# #6246 Initialize should be composed of Resets + StatePreparation

### Motivation

- Existing `Initialize` class adds a reset followed by some state preparation functionality (differs for int, string, statevector etc.)
- Users want to be able to prepare state without automatically adding a reset, use more `Gate` functions (`Initialize` is an Instruction, can't inverse, add controls etc.)
- Original GitHub issue:https://github.com/Qiskit/qiskit-terra/issues/6246
- Stack Exchange: https://quantumcomputing.stackexchange.com/questions/24535/is-it-possible-to-create-a-controlled-initialize-instruction-in-qiskit

### New StatePreparation Class

TL;DR 

a `Gate` that does everything `Initialize` used to do, without adding a reset

`Initialize` now adds a reset calls `StatePreparation` under the hood

Users can continue to use `Initialize` the same as before

### Imports

In [64]:
import numpy as np
import math
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister

from qiskit.extensions.quantum_initializer import Initialize
from qiskit.circuit.library import StatePreparation

### Initialize Implementation

In [17]:
qr = QuantumRegister(2)
qc_init = QuantumCircuit(qr)

# int param
init = Initialize(2)

# string param
# init = Initialize('11')

# statevector param
# init = Initialize([1 / math.sqrt(2), 0, 0, 1 / math.sqrt(2)])

qc_init.append(init, qr)

qc_init.draw()

┌──────────────────────────────────┐
q8_0: ┤0                                 ├
      │  Initialize(0.70711,0,0,0.70711) │
q8_1: ┤1                                 ├
      └──────────────────────────────────┘

In [18]:
qc_init.decompose().draw()

┌─────────────────────────────────────────┐
q8_0: ─|0>─┤0                                        ├
           │  State Preparation(0.70711,0,0,0.70711) │
q8_1: ─|0>─┤1                                        ├
           └─────────────────────────────────────────┘

In [19]:
qc_init.decompose().decompose().draw()

┌──────────────────┐
q8_0: ─|0>─┤0                 ├
           │  disentangler_dg │
q8_1: ─|0>─┤1                 ├
           └──────────────────┘

In [21]:
qc_init2 = QuantumCircuit(2)

qc_init2.initialize(2)

qc_init2.draw()

┌────────────────┐
q_0: ┤0               ├
     │  Initialize(2) │
q_1: ┤1               ├
     └────────────────┘

### StatePreparation Class Implementation

In [53]:
circ = QuantumCircuit(2)

# int param
# my_state = StatePreparation(2, label="my state")

# string param
# my_state = StatePreparation('11', label="my state")

# statevector param
# my_state = StatePreparation([1 / math.sqrt(2), 0, 0, 1 / math.sqrt(2)], label="my state")

circ.append(my_state, [0,1])

circ.draw()

┌────────────────────────────────┐
q_0: ┤0                               ├
     │  my state(0.70711,0,0,0.70711) │
q_1: ┤1                               ├
     └────────────────────────────────┘

In [54]:
circ.decompose().draw()

┌──────────────────┐
q_0: ┤0                 ├
     │  disentangler_dg │
q_1: ┤1                 ├
     └──────────────────┘

### StatePreparation inverse

In [55]:
circ.append(my_state.inverse(), [0,1])
circ.draw()

┌────────────────────────────────┐┌───────────────────────────────────┐
q_0: ┤0                               ├┤0                                  ├
     │  my state(0.70711,0,0,0.70711) ││  my state Dg(0.70711,0,0,0.70711) │
q_1: ┤1                               ├┤1                                  ├
     └────────────────────────────────┘└───────────────────────────────────┘

In [56]:
circ.decompose().draw()

┌──────────────────┐┌───────────────┐
q_0: ┤0                 ├┤0              ├
     │  disentangler_dg ││  disentangler │
q_1: ┤1                 ├┤1              ├
     └──────────────────┘└───────────────┘

### StatePreparation Control

In [72]:
qc_control = QuantumCircuit(3)

stateprep = StatePreparation('11')
controlled_stateprep = stateprep.control(1)

qc_control.append(controlled_stateprep, [0,1,2])

qc_control.draw()

q_0: ─────────────■─────────────
     ┌────────────┴────────────┐
q_1: ┤0                        ├
     │  State Preparation(1,1) │
q_2: ┤1                        ├
     └─────────────────────────┘

### prepare_state() a la initialize()

In [58]:
qc = QuantumCircuit(2)

# int param
# qc.prepare_state(2, label="my cool state")

# string param
# qc.prepare_state('11', label="my cool state")

# statevector param
qc.prepare_state([1 / math.sqrt(2), 0, 0, 1 / math.sqrt(2)], label="my cool state")

qc.draw()

┌─────────────────────────────────────┐
q_0: ┤0                                    ├
     │  my cool state(0.70711,0,0,0.70711) │
q_1: ┤1                                    ├
     └─────────────────────────────────────┘

## Potential Areas of Further Development

- Could allow users to specify synthesis (e.g. could call `Isometry`)
- Currently `StatePreparation` assumes starting in |0> state

In [5]:
qc = QuantumCircuit(2)
# qc.t(0)
qc.prepare_state([0,1,0,0], label="my cool state")
qc.draw()

TypeError: prepare_state() got an unexpected keyword argument 'label'

In [4]:
qc_dg = qc.inverse().inverse().inverse().inverse()
qc_dg.draw()

# qc_dg.data

┌─────────────────────────────┐
q_0: ┤0                            ├
     │  State Preparation(0,1,0,0) │
q_1: ┤1                            ├
     └─────────────────────────────┘

In [7]:
my_gate = StatePreparation(2)
print(my_gate.params)
my_gate_dg = my_gate.inverse()

[(2+0j)]


QiskitError: 'Desired statevector length not a positive power of 2.'

In [5]:
g = TGate()
dg = g.inverse
dg

<bound method TGate.inverse of Instruction(name='t', num_qubits=1, num_clbits=0, params=[])>

In [7]:
circ_dg = QuantumCircuit(2)
circ_dg.append(my_gate_dg, [0,1])
circ_dg.draw()

TypeError: issubclass() arg 1 must be a class

### test mutating params

In [9]:
qr = QuantumRegister(2)
qc = QuantumCircuit(qr)
qc.append(init, qr)
qc.decompose().draw()

┌─────────────────────────┐
q0_0: ─|0>─┤0                        ├
           │  State Preparation(1,1) │
q0_1: ─|0>─┤1                        ├
           └─────────────────────────┘

In [10]:
init.params = '00'
print(init.params)
print(init)

['0', '0']
Instruction(name='initialize', num_qubits=2, num_clbits=0, params=['0', '0'])


In [12]:
stateprep_init = StatePreparation('11')
print(stateprep_init.params)

['1', '1']


In [13]:
qr = QuantumRegister(2)
qc = QuantumCircuit(qr)
qc.append(stateprep_init, qr)
qc.draw()
# qc.decompose().draw()

┌─────────────────────────┐
q2_0: ┤0                        ├
      │  State Preparation(1,1) │
q2_1: ┤1                        ├
      └─────────────────────────┘

In [15]:
stateprep_init.params = '10'
stateprep_init.params

['1', '0']

In [16]:

qr = QuantumRegister(2)
qc = QuantumCircuit(qr)
qc.append(stateprep_init, qr)
qc.draw()
# qc.decompose().draw()

┌─────────────────────────┐
q6_0: ┤0                        ├
      │  State_preparation(1,0) │
q6_1: ┤1                        ├
      └─────────────────────────┘

In [40]:
# qr = QuantumRegister(2)
qc = QuantumCircuit(3)
qc.prepare_state('11')
qc.draw()
qc.decompose().draw()

QiskitError: 'StatePreparation parameter vector has 4 elements, therefore expects 2 qubits. However, 3 were provided.'

In [25]:
qr = QuantumRegister(2)
qr2 = QuantumRegister(1)
qc = QuantumCircuit(qr, qr2)
# qc.initialize('11')
# qc.initialize(2) #BROKEN
qc.initialize([0,1/np.sqrt(2),1/np.sqrt(2),0], [0,1])
qc.inverse()
qc.draw()
# _define never called??

CircuitError: "Invalid param type <class 'complex'> for gate initialize_dg."

In [10]:
qc.data

[(Instruction(name='initialize', num_qubits=2, num_clbits=0, params=[0j, (0.7071067811865475+0j), (0.7071067811865475+0j), 0j]), [Qubit(QuantumRegister(2, 'q4'), 0), Qubit(QuantumRegister(2, 'q4'), 1)], [])]

In [11]:
Initialize.params('01')

TypeError: 'property' object is not callable

In [12]:
dc = qc.decompose()
dc.draw()

┌─────────────────────────────────────────┐
q4_0: ─|0>─┤0                                        ├
           │  State_preparation(0,0.70711,0.70711,0) │
q4_1: ─|0>─┤1                                        ├
           └─────────────────────────────────────────┘
  q5: ────────────────────────────────────────────────

In [13]:
dc.data

[(Instruction(name='reset', num_qubits=1, num_clbits=0, params=[]), [Qubit(QuantumRegister(2, 'q4'), 0)], []), (Instruction(name='reset', num_qubits=1, num_clbits=0, params=[]), [Qubit(QuantumRegister(2, 'q4'), 1)], []), (Instruction(name='state_preparation', num_qubits=2, num_clbits=0, params=[0j, (0.7071067811865475+0j), (0.7071067811865475+0j), 0j]), [Qubit(QuantumRegister(2, 'q4'), 0), Qubit(QuantumRegister(2, 'q4'), 1)], [])]

In [27]:
circuit = QuantumCircuit(2)
circuit.prepare_state('01', circuit.qubits)
# qc.inverse()
circuit.draw()

┌───┐┌─────────────────────────┐
q_0: ┤ H ├┤0                        ├
     └───┘│  State_preparation(0,1) │
q_1: ─────┤1                        ├
          └─────────────────────────┘

In [9]:
qc_state = QuantumCircuit(2)
qc_state.h(0)
# qc_state.prepare_state('10')
# qc_state.prepare_state(2) #BROKEN
qc_state.prepare_state([0,1/np.sqrt(2),1/np.sqrt(2),0])
# qc_state.prepare_state([0,1,0,0])
qc_state.inverse()
qc_state.draw()

┌───┐┌─────────────────────────────────────────┐
q_0: ┤ H ├┤0                                        ├
     └───┘│  State Preparation(0,0.70711,0.70711,0) │
q_1: ─────┤1                                        ├
          └─────────────────────────────────────────┘

In [31]:
print(qc_state.decompose())

     ┌─────────┐┌──────────────────┐
q_0: ┤ U2(0,π) ├┤0                 ├
     └─────────┘│  disentangler_dg │
q_1: ───────────┤1                 ├
                └──────────────────┘


In [17]:
qc_state.data

[(Instruction(name='state_preparation', num_qubits=2, num_clbits=0, params=['1', '1']), [Qubit(QuantumRegister(2, 'q'), 0), Qubit(QuantumRegister(2, 'q'), 1)], [])]

In [18]:
qr = QuantumRegister(2)
qc = QuantumCircuit(qr, global_phase=0.5)
qc.h(0)
qc.barrier(qr)
qc.t(1)
# qc.inverse()
qc.draw()

# expected = QuantumCircuit(qr)
# expected.tdg(1)
# expected.barrier(qr)
# expected.h(0)
# expected.global_phase = -0.5
# self.assertEqual(qc.inverse(), expected)

global phase: 0.5
      ┌───┐ ░      
q6_0: ┤ H ├─░──────
      └───┘ ░ ┌───┐
q6_1: ──────░─┤ T ├
            ░ └───┘